In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast

In [2]:
error

NameError: name 'error' is not defined

In [ ]:
# size_width_rugs_runners
# size_length_rugs_runners
# size_diameter_rugs_runners

# parameters
customer_id = '5'
formatted_attribute = 'size_length_rugs_runners'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Rugs & Mats"""


# regex_pattern = r"(?i)(\s[^t]\d+\-Inch x \d+\-Inch)|()"
regex_pattern = r"(?i)(\s[^t]\d+\-Inch x \d+\-Inch)|(\d+\\\" x \d*\\\")"


value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [ ]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [ ]:
df = query_from_file(file_name='query/bbby.sql', params=params)
df

In [ ]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

In [ ]:
metric=""
range_string =""""
 0 - 11.9
 12 - 23.9
 24 - 35.9
 36 - 47.9
 48 - 59.9
 60 - 71.9
 72 - 83.9
 84 - 95.9
 96 - 107.9
 Greater than 108
 """

In [ ]:
# kwp=KeywordProcessor()
# kwp.add_keywords_from_list(list_values)
# print(kwp.add_keywords_from_list(list_values))
## Extract Key Words
# valuez=x['Max'].apply(lambda x: kwp.extract_keywords(x))
# valuez[0:50]

In [ ]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [ ]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df

In [ ]:
# df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
# df

In [ ]:
x=df[df['matches'].astype(str)!='[]']
x

In [ ]:
x['matches']=x['matches'].str.get(0)
x['matches']=x['matches'].apply(lambda x:x.lower())
x[['length', 'width']]=x['matches'].str.split(" x ",expand=True,)
x

In [ ]:
x['length']=x['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\"','',str(measure)))
x['width']=x['width'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\"','',str(measure)))
x['length']=x['length'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
x['width']=x['width'].apply(lambda quote: quote.replace('"', ''))
x['length']=x['length'].replace('"', '').apply(lambda quote: quote.replace('-',''))
x['width']=x['width'].apply(lambda quote: quote.replace('-',''))
x['length']=x['length'].astype(int)
x['width']=x['width'].astype(int)
x

In [ ]:
x['Max'] = x[['length', 'width']].values.max(1)
x['Min'] = x[['length', 'width']].values.min(1)
x

In [ ]:
x[0:50]

In [ ]:
# x['Max']=x['Max'].astype(str)
# x['Final']=x['Max'].apply(lambda x: kwp.extract_keywords(x))
# x

In [ ]:
matches=x
matches

In [ ]:
string = range_string
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return f'{lower}" - {upper}"{metric}'
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)

In [ ]:
# matches['Min']=matches['Min'].astype(int)
# matches['final']=matches['Max'].apply(lambda x: kwp.extract_keywords(x))
# matches.dtypes

In [ ]:
matches[curation_col]=matches['Max'].apply(range_app)
matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

In [ ]:
matches[matches['Q:size_length_rugs_runners']=='n/a']

In [ ]:
# error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na {attribute} {buckets} Shawn.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match {attribute} {buckets} Shawn.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, matches)